# Solution for Assignment 2: Quantity Discount Models

This notebook solves the problems from Assignment 2 using the `imf_*` formula library.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import math

# Get the absolute path of the current file and add parent directory to path
current_dir = os.path.dirname(os.path.abspath('__file__'))
parent_dir = os.path.dirname(current_dir)
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

# Now we can import from imf_main
try:
    from imf_main import eoq_all_unit_quantity_discount, eoq_incremental_quantity_discount
except ImportError:
    print("Could not import from imf_main. Using direct implementation...")
    
    # Direct implementation of quantity discount functions if import fails
    def eoq_all_unit_quantity_discount(demand, setup_cost, unit_costs, holding_rate, quantity_breaks):
        """Calculate EOQ with all-unit quantity discount"""
        candidates = []
        
        # For each price point
        for i in range(len(unit_costs)):
            holding_cost = holding_rate * unit_costs[i]
            q = math.sqrt(2 * demand * setup_cost / holding_cost)
            
            lower_bound = quantity_breaks[i]
            upper_bound = quantity_breaks[i+1] if i+1 < len(quantity_breaks) else float('inf')
            
            if lower_bound <= q < upper_bound:
                # Feasible EOQ
                ordering_cost = setup_cost * demand / q
                inventory_cost = holding_cost * q / 2
                purchase_cost = unit_costs[i] * demand
                total_cost = ordering_cost + inventory_cost + purchase_cost
                candidates.append({'order_quantity': q, 'unit_cost': unit_costs[i], 
                                   'total_cost': total_cost, 'ordering_cost': ordering_cost, 
                                   'holding_cost': inventory_cost, 'purchase_cost': purchase_cost})
            elif q < lower_bound and i < len(unit_costs) - 1:
                # Check next price point
                continue
            else:
                # Use break point
                q = lower_bound
                ordering_cost = setup_cost * demand / q
                inventory_cost = holding_cost * q / 2
                purchase_cost = unit_costs[i] * demand
                total_cost = ordering_cost + inventory_cost + purchase_cost
                candidates.append({'order_quantity': q, 'unit_cost': unit_costs[i], 
                                   'total_cost': total_cost, 'ordering_cost': ordering_cost, 
                                   'holding_cost': inventory_cost, 'purchase_cost': purchase_cost})
        
        # Find minimum cost option
        return min(candidates, key=lambda x: x['total_cost'])
    
    def eoq_incremental_quantity_discount(demand, setup_cost, unit_costs, holding_rate, quantity_breaks):
        """Calculate EOQ with incremental quantity discount"""
        # This is a simplified implementation
        breaks = [0] + quantity_breaks
        candidates = []
        
        for j in range(1, len(unit_costs) + 1):
            # Calculate average unit cost at various quantities
            for q in np.linspace(breaks[j-1], breaks[j-1] + 500, 10):
                # Calculate purchase cost with incremental pricing
                purchase_cost = 0
                for i in range(1, j):
                    purchase_cost += unit_costs[i-1] * (breaks[i] - breaks[i-1])
                purchase_cost += unit_costs[j-1] * (q - breaks[j-1])
                purchase_cost *= demand / q
                
                # Calculate holding cost
                avg_cost = purchase_cost / demand
                holding_cost = holding_rate * avg_cost
                
                # Calculate ordering cost
                ordering_cost = setup_cost * demand / q
                
                # Total cost
                total_cost = ordering_cost + holding_cost * q/2 + purchase_cost
                
                candidates.append({'order_quantity': q, 'price_range': j, 
                                   'total_cost': total_cost, 'ordering_cost': ordering_cost, 
                                   'holding_cost': holding_cost * q/2, 'purchase_cost': purchase_cost})
        
        # Find minimum cost option
        return min(candidates, key=lambda x: x['total_cost'])

## Exercise 2(a): All-unit quantity discount model

In [ ]:
# Given data
demand = 40 * 52  # units/year
setup_cost = 25  # euro
interest_rate = 0.26  # holding cost as fraction of unit cost

# Break points of order quantity and corresponding purchasing prices
quantity_breaks = [0, 300, 500]
unit_costs = [10, 9.7, 9.5]

# Calculate optimal order quantity using the library function
result = eoq_all_unit_quantity_discount(
    demand=demand,
    setup_cost=setup_cost,
    unit_costs=unit_costs,
    holding_rate=interest_rate,
    quantity_breaks=quantity_breaks
)

print(f"Optimal order quantity: {result['order_quantity']:.2f} units")
print(f"Optimal unit cost: {result['unit_cost']:.2f} euro")
print(f"Total annual cost: {result['total_cost']:.2f} euro")
print(f"Ordering cost: {result['ordering_cost']:.2f} euro")
print(f"Holding cost: {result['holding_cost']:.2f} euro")
print(f"Purchase cost: {result['purchase_cost']:.2f} euro")

## Exercise 2(b): Incremental quantity discount model

In [ ]:
# Given data (simplified for incremental quantity discount)
demand = 40 * 52  # units/year
setup_cost = 25  # euro
interest_rate = 0.26  # holding cost as fraction of unit cost

# Simplified case with just two price points for incremental discount
quantity_breaks = [300]
unit_costs = [10, 9.7]

# Calculate optimal order quantity using the library function
result = eoq_incremental_quantity_discount(
    demand=demand,
    setup_cost=setup_cost,
    unit_costs=unit_costs,
    holding_rate=interest_rate,
    quantity_breaks=quantity_breaks
)

print(f"Optimal order quantity: {result['order_quantity']:.2f} units")
print(f"Optimal price range: {result['price_range']}")
print(f"Total annual cost: {result['total_cost']:.2f} euro")
print(f"Ordering cost: {result['ordering_cost']:.2f} euro")
print(f"Holding cost: {result['holding_cost']:.2f} euro")
print(f"Purchase cost: {result['purchase_cost']:.2f} euro")

## Detailed Step-by-Step Approach for All-Unit Quantity Discount

Let's also implement the step-by-step approach for solving the all-unit quantity discount model to better understand the calculations:

In [ ]:
import math

# Given data
d = 40 * 52  # unit/year
A = 25  # euro
r = 0.26  # euro/year #interest rate

bp = [0, 300, 500]  # break points of order quantity
cp = [10, 9.7, 9.5]  # purchasing prices

# Step 1) Calculate EOQ for the discounted price:
t = len(cp) - 1  # start from the cheapest price
holding_cost = r * cp[t]
qt = math.sqrt(2 * A * d / holding_cost)
print(f"EOQ at price level {t} (${cp[t]}): {qt:.2f} units")

# Step 2) Check if EOQ is feasible
if qt >= bp[t]:
    q_opt = qt
    c_opt = d / qt * A + 0.5 * holding_cost * qt + cp[t] * d
    print(f"EOQ at level {t} is feasible")
else:
    print(f"EOQ at level {t} is infeasible")
    
    # Step 3) Calculate EOQ for the less favorable price
    while t >= 1 and qt < bp[t]:
        t -= 1
        holding_cost = r * cp[t]
        qt = math.sqrt(2 * A * d / holding_cost)
        print(f"EOQ at price level {t} (${cp[t]}): {qt:.2f} units")
        
        # Calculate cost at the break point of the next level
        cost_break = d / bp[t + 1] * A + 0.5 * r * cp[t + 1] * bp[t + 1] + cp[t + 1] * d
        print(f"Cost at break point {bp[t+1]}: {cost_break:.2f} euro")
        
        # Calculate cost at EOQ for current level
        cost_eoq = d / qt * A + 0.5 * holding_cost * qt + cp[t] * d
        print(f"Cost at EOQ {qt:.2f}: {cost_eoq:.2f} euro")
        
        # Step 4) compare cost at the break point and at EOQ
        if cost_break < cost_eoq:
            q_opt = bp[t + 1]
            c_opt = cost_break
            print(f"Breakpoint {bp[t+1]} preferred")
            break
        else:
            print(f"Lower EOQ at level {t} preferred")
            q_opt = qt
            c_opt = cost_eoq
            if qt >= bp[t]:
                print(f"EOQ at level {t} feasible")
            else:
                print(f"EOQ at level {t} infeasible")

print(f"\nOptimal order quantity: {q_opt:.2f} units")
print(f"Optimal total cost: {c_opt:.2f} euro")

## Detailed Step-by-Step Approach for Incremental Quantity Discount

In [ ]:
# Given data
d = 40 * 52  # unit/year
setup_cost = 25  # euro
r = 0.26  # euro/year

bp = [0, 300]
cp = [10, 9.7]

# Step 1) Compute the sum of terms that are independent of Q in purchasing cost
R = np.zeros(len(bp))
for t in range(1, len(bp)):
    R[t] = cp[t-1] * (bp[t] - bp[t-1]) + R[t-1]
    
print(f"R values: {R}")

# Step 2) Compute EOQ for all segments & Check the feasibility of EOQs
qt = np.zeros(len(bp))
flag_feasible = np.full(len(bp), False, dtype=bool)  # the indicator of feasibility

for t in range(len(bp)):
    qt[t] = math.sqrt(2 * (R[t] - cp[t] * bp[t] + setup_cost) * d / (r * cp[t]))
    print(f"EOQ for segment {t}: {qt[t]:.2f}")

for t in range(len(bp) - 1):
    flag_feasible[t] = True if qt[t] >= bp[t] and qt[t] < bp[t+1] else False
flag_feasible[-1] = True if qt[-1] >= bp[-1] else False

print(f"EOQ is feasible: {flag_feasible}")

# Step 3) Compute the total cost function for feasible EOQs
def lot_cost(q, h, c):
    return d / q * setup_cost + 0.5 * h * q + c * d

qt_f = []
cost_qt_f = []

for t in range(len(bp)):
    if flag_feasible[t]:
        q = qt[t]
        c = (R[t] + cp[t] * (q - bp[t])) / q
        holding_cost = c * r
        cost_q = lot_cost(q, holding_cost, c)
        qt_f.append(q)
        cost_qt_f.append(cost_q)
        print(f"Segment {t}, q={q:.2f}, cost={cost_q:.2f}")

c_opt = min(cost_qt_f)
q_opt = qt_f[np.argmin(cost_qt_f)]

print(f"\nOptimal order quantity: {q_opt:.2f} units")
print(f"Optimal total cost: {c_opt:.2f} euro")

## Visualization Comparing All-Unit and Incremental Quantity Discount Models

In [ ]:
# Define quantity range
q_values = np.linspace(100, 700, 1000)

# All-unit quantity discount
q_breaks = [0, 300, 500]
unit_costs = [10, 9.7, 9.5]

# Calculate total cost for all-unit quantity discount
all_unit_costs = []
for q in q_values:
    # Determine the unit cost based on order quantity
    if q < 300:
        c = unit_costs[0]
    elif q < 500:
        c = unit_costs[1]
    else:
        c = unit_costs[2]
    
    holding_cost = interest_rate * c
    ordering_cost = setup_cost * demand / q
    inventory_cost = holding_cost * q / 2
    purchase_cost = c * demand
    
    total_cost = ordering_cost + inventory_cost + purchase_cost
    all_unit_costs.append(total_cost)

# Incremental quantity discount
q_breaks_inc = [0, 300]
unit_costs_inc = [10, 9.7]

# Calculate total cost for incremental quantity discount
incremental_costs = []
for q in q_values:
    # Calculate purchase cost with incremental pricing
    if q <= 300:
        c = unit_costs_inc[0]
        holding_cost = interest_rate * c
        purchase_cost = c * demand
    else:
        # First 300 units at higher price, remaining at lower price
        purchase_cost = (unit_costs_inc[0] * 300 + unit_costs_inc[1] * (q - 300)) * demand / q
        # Calculate average cost for holding cost
        avg_cost = purchase_cost / demand
        holding_cost = interest_rate * avg_cost
    
    ordering_cost = setup_cost * demand / q
    inventory_cost = holding_cost * q / 2
    
    total_cost = ordering_cost + inventory_cost + purchase_cost
    incremental_costs.append(total_cost)

# Create plot
plt.figure(figsize=(10, 6))
plt.plot(q_values, all_unit_costs, label='All-unit discount')
plt.plot(q_values, incremental_costs, label='Incremental discount')

# Mark price breaks
for q_break in q_breaks:
    if q_break > 0:  # Skip the 0 break point
        plt.axvline(x=q_break, color='r', linestyle='--', alpha=0.5)
        plt.text(q_break, max(max(all_unit_costs), max(incremental_costs)) * 0.9, 
                 f'Q = {q_break}', rotation=90)

# Add labels and legend
plt.xlabel('Order Quantity')
plt.ylabel('Total Annual Cost')
plt.title('Comparison of Quantity Discount Models')
plt.grid(True, alpha=0.3)
plt.legend()

plt.show()